In [186]:
import pprint
import pymongo
from pymongo import MongoClient,GEO2D,GEOSPHERE,InsertOne, DeleteOne, ReplaceOne
from datetime import datetime,time
from bson.son import SON
from collections import OrderedDict
client = MongoClient('mongodb://localhost:27017/')
#client = MongoClient('localhost', 27017)

<b>Creating and working with GeoSpatial Index </b>

In [187]:
db=client.mydb

In [188]:
Collections=db.geolocations

<b> 2D INDEXES </b>

In [189]:
Collections.insert_many([{"Resturant":"raising_caine","address":"Westheimer","loc":[45,78]},
                   {"Resturant":"Chickafilla","address":"place""West_bellfort","loc":[14,45]},
                   {"Resturant":"Mc_Donald","address":"stafford","loc":[49,72]},
                   {"Resturant":"Ceasar","address":"Beechnut","loc":[87,75]}
                        ])

In [190]:
Collections.find_one({"Resturant":"Ceasar"})

{'_id': ObjectId('5fbfdd96fc872af180c549c8'),
 'Resturant': 'Ceasar',
 'address': 'Beechnut',
 'loc': [87, 75]}

In [191]:
Collections.find_one({"Resturant":"Ceasar"})

{'_id': ObjectId('5fbfdd96fc872af180c549c8'),
 'Resturant': 'Ceasar',
 'address': 'Beechnut',
 'loc': [87, 75]}

In [192]:
Collections.find({"Resturant":"Ceasar"})

In [193]:
#Creating a Geo2d which use for flat location points

Collections.create_index([("loc",GEO2D)])

'loc_2d'

In [194]:
pprint.pprint(Collections.find_one({"loc":{"$near":[45,78]}}))

{'Resturant': 'raising_caine',
 '_id': ObjectId('5fbfdd96fc872af180c549c5'),
 'address': 'Westheimer',
 'loc': [45, 78]}


In [195]:
# The SON/OrederDict is use to order the dictionary

for loc in Collections.find({"loc":OrderedDict([("$near",[45,78]),("$maxDistance",5)])}).limit(3):
    print(loc)

{'_id': ObjectId('5fbfdd96fc872af180c549c5'), 'Resturant': 'raising_caine', 'address': 'Westheimer', 'loc': [45, 78]}
{'_id': ObjectId('5fbfde0afc872af180c549ce'), 'Resturant': 'raising_caine', 'address': 'Westheimer', 'loc': [45, 78]}
{'_id': ObjectId('5fbff4f7fc872af180c549e7'), 'Resturant': 'raising_caine', 'address': 'Westheimer', 'loc': [45, 78]}


In [196]:
Collections.find_one(
   {"loc" : {"$near" : [ 45, 78 ], "$maxDistance": 0.10 } }
)

{'_id': ObjectId('5fbfdd96fc872af180c549c5'),
 'Resturant': 'raising_caine',
 'address': 'Westheimer',
 'loc': [45, 78]}

<b> Coordinating Lying Within a Boundary</b>

In [197]:
# Find a location with in a defined Box
Collections.find_one({
  "loc": {
     "$geoWithin": {
       "$box": [[0,0],[100,100]]
     }
  }
})

{'_id': ObjectId('5fbfdd96fc872af180c549c6'),
 'Resturant': 'Chickafilla',
 'address': 'placeWest_bellfort',
 'loc': [14, 45]}

In [198]:
# Find a location within a defined Center
Collections.find_one(
   { "loc": {"$geoWithin": {"$center": [ [45, 78], 10 ] } } }
)

{'_id': ObjectId('5fbfdd96fc872af180c549c7'),
 'Resturant': 'Mc_Donald',
 'address': 'stafford',
 'loc': [49, 72]}

In [199]:
# Find a location within a defined Center
Collections.find_one(
   { "loc": {"$geoWithin": {"$center": [ [45, 78], 10 ] } } }
)

{'_id': ObjectId('5fbfdd96fc872af180c549c7'),
 'Resturant': 'Mc_Donald',
 'address': 'stafford',
 'loc': [49, 72]}

<b> 2D INDEXES SPHERE </b>

A 2dsphere index supports queries that calculate geometries on an earth-like sphere. 2dsphere index supports

In [200]:
db.Collection2.insert(
   {
      "loc" : { "type": "Point", "coordinates": [ -73.97, 40.77 ] },
      "name": "Central Park",
      "category" : "Parks"
   }
)

db.Collection2.insert(
   {
      "loc" : { "type": "Point", "coordinates": [ -73.88, 40.78 ] },
      "name": "La Guardia Airport",
      "category" : "Airport"
   }
)

C:\Users\00936124\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """
C:\Users\00936124\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  del sys.path[0]


ObjectId('5fc02638fc872af180c549f4')

In [201]:
db.Collection2.insert(
   {
      "loc" : { "type": "Point", "coordinates": [ -73.88, 40.78 ] },
      "name": "La Guardia Airport",
      "category" : "Airport"
   }
)

C:\Users\00936124\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
  """


ObjectId('5fc02639fc872af180c549f5')

In [202]:
list(db.Collection2.find())

[{'_id': ObjectId('5fbff411fc872af180c549e2'),
  'loc': {'type': 'Point', 'coordinates': [-73.97, 40.77]},
  'name': 'Central Park',
  'category': 'Parks'},
 {'_id': ObjectId('5fbff483fc872af180c549e5'),
  'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]},
  'name': 'La Guardia Airport',
  'category': 'Airport'},
 {'_id': ObjectId('5fbff4fafc872af180c549eb'),
  'loc': {'type': 'Point', 'coordinates': [-73.97, 40.77]},
  'name': 'Central Park',
  'category': 'Parks'},
 {'_id': ObjectId('5fbff4fafc872af180c549ec'),
  'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]},
  'name': 'La Guardia Airport',
  'category': 'Airport'},
 {'_id': ObjectId('5fbff4fafc872af180c549ed'),
  'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]},
  'name': 'La Guardia Airport',
  'category': 'Airport'},
 {'_id': ObjectId('5fc01fe91e3e277019e355b6'),
  'Name': 'Deer Park',
  'category': 'Park',
  'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}},
 {'_id': ObjectId('5fc02638fc872af180c

<b>Creating Index </b>

In [203]:
db.Collection2.create_index([("loc",GEOSPHERE)])

'loc_2dsphere'

In [204]:
for i in db.Collection2.find( {"loc" :
                         SON([("$near",
                           { "$geometry" :
                             SON([("type", "Point") ,
                               ("coordinates", [-73.01, 40.1])])})])}):
    print(i)

{'_id': ObjectId('5fbff483fc872af180c549e5'), 'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}, 'name': 'La Guardia Airport', 'category': 'Airport'}
{'_id': ObjectId('5fbff4fafc872af180c549ed'), 'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}, 'name': 'La Guardia Airport', 'category': 'Airport'}
{'_id': ObjectId('5fc01fe91e3e277019e355b6'), 'Name': 'Deer Park', 'category': 'Park', 'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}}
{'_id': ObjectId('5fc02639fc872af180c549f5'), 'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}, 'name': 'La Guardia Airport', 'category': 'Airport'}
{'_id': ObjectId('5fbff4fafc872af180c549ec'), 'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}, 'name': 'La Guardia Airport', 'category': 'Airport'}
{'_id': ObjectId('5fc02638fc872af180c549f4'), 'loc': {'type': 'Point', 'coordinates': [-73.88, 40.78]}, 'name': 'La Guardia Airport', 'category': 'Airport'}
{'_id': ObjectId('5fc02638fc872af180c549f3'), 'loc': {'type': 'Point',

<b>Upserting</b>

With the upsert option set to true, if no matching documents exist for the Bulk.find() condition,
then the update or the replacement operation performs an insert. If a matching document does exist, then the update or 
replacement operation performs the specified update or replacement.

In [230]:
requests = [InsertOne({"loc":{ "type": "Point", "coordinates": [ -73.89, 50.78 ] },"Name": "pasadena Park",\
                       "category":"Park"}),ReplaceOne({'Name':"Deer Park"}, {"Name":'Downtown Park'}, upsert=True)]

db.Collection2.bulk_write(requests,ordered=True) 


In [231]:
for i in db.Collection2.find():
    print(i)

{'_id': ObjectId('5fc03f69fc872af180c54a03'), 'loc': {'type': 'Point', 'coordinates': [-73.89, 50.78]}, 'Name': 'pasadena Park', 'category': 'Park'}
{'_id': ObjectId('5fc03f691e3e277019e38763'), 'Name': 'Downtown Park'}


In [207]:
db=client.db
Students=db.cStudents

In [209]:
Students.find_one()

{'_id': ObjectId('5fbd61ede6c5176e25b21d18'),
 'i': 2.0,
 'age': 1.0,
 'username': 'user2'}

In [216]:
#Bulikng insertion
bulk = Students.initialize_ordered_bulk_op()
bulk.find({"i": 2.0 } ).upsert().update({"$inc": {"i": 5,"age":10}});
bulk.execute();

C:\Users\00936124\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: initialize_ordered_bulk_op is deprecated
  


In [217]:
Students.find_one()

{'_id': ObjectId('5fbd61ede6c5176e25b21d18'),
 'i': 7.0,
 'age': 11.0,
 'username': 'user2'}

<b>#Catching and Error</b>

In [233]:

from pymongo.errors import BulkWriteError

bulk = Students.initialize_unordered_bulk_op()
bulk.insert({"i":1001})
bulk.insert({"i":1002})
bulk.find({"i":2}).remove_one()
bulk.find({"i":7.0}).replace_one({"i":1001})
           
try:
    bulk.execute({"w":1,"wtimeout":1})
except BulkWriteError as bwe:
    print(bwe.deatils)

C:\Users\00936124\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: initialize_unordered_bulk_op is deprecated
  This is separate from the ipykernel package so we can avoid doing imports until


<b> Using GridFS</b>

In [253]:
import gridfs
fs=gridfs.GridFS(db)
mymusic=fs.put(b"test_image.jpg")

# 
res=fs.get(mymusic).read()
print(res)

b'test_image.jpg'


In [256]:
b = fs.put(fs.get(mymusic), filename="foo", bar="baz")
out = fs.get(b)
print(out)

In [257]:
out.filename

'foo'

In [258]:
out.upload_date

datetime.datetime(2020, 11, 27, 0, 48, 57, 420000)

<b> Better Explanation of GridFs</b>

In [305]:
import os
fileID=print(os.path.getsize(r"C:\Users\c_seromonsei\Pricing Review.mp4"))

438393593


In [308]:
#Putting and getting the it into db the GridFS split file into Chunks fs.Chunks,fs.files
meetvideos=fs.put(b"C:\Users\c_seromonsei\Pricing Review.mp4")
out = fs.get(meetvideos).read()

In [344]:
b = fs.put(b"C:\Users\c_seromonsei\Pricing Review.mp4", filename="meetvideos")
out = fs.get(b)
print(out.read(),out.filename,out.uploadDate,out.length)

b'C:\\Users\\c_seromonsei\\Pricing Review.mp4' meetvideos 2020-11-27 02:14:05.650000 40


In [325]:
#res=fs.get(fs.put(b"C:\Users\c_seromonsei\Pricing Review.mp4")).read()
#print(res)

In [331]:
for i in db.fs.files.find({"filename":"meetvideos"}):
    print(i)

{'_id': ObjectId('5fc05d31fc872af180c54a3f'), 'filename': 'meetvideos', 'md5': 'a169fabe2bd9578e20154ebade59e7e0', 'chunkSize': 261120, 'length': 40, 'uploadDate': datetime.datetime(2020, 11, 27, 1, 58, 9, 641000)}
{'_id': ObjectId('5fc05dd3fc872af180c54a41'), 'filename': 'meetvideos', 'md5': 'a169fabe2bd9578e20154ebade59e7e0', 'chunkSize': 261120, 'length': 40, 'uploadDate': datetime.datetime(2020, 11, 27, 2, 0, 51, 352000)}
{'_id': ObjectId('5fc05de9fc872af180c54a43'), 'filename': 'meetvideos', 'md5': 'a169fabe2bd9578e20154ebade59e7e0', 'chunkSize': 261120, 'length': 40, 'uploadDate': datetime.datetime(2020, 11, 27, 2, 1, 13, 737000)}


In [343]:
db.fs.chunks.createIndex( {"_id": 1 },{ "unique": True })

TypeError: 'Collection' object is not callable. If you meant to call the 'createIndex' method on a 'Collection' object it is failing because no such method exists.

In [342]:
db.fs.chunks.find({"_id":"5fc05d31fc872af180c54a3f"})